# LAB 9 : Algoritmo Forward Backward en HMM

In [44]:
'''Definir variables y datos iniciales'''

# posibles estados
states = ['sunny', 'rainy']

# observaciones previas
observations = ['sunny', 'sunny', 'rainy']
print(f"observations : {observations}")

# probabilidades iniciales
initial_prob = {'sunny':0.5 , 'rainy': 0.5}
print(f"initial probability : {initial_prob}")

# probabilidades de transiciones

sunny_transition = {'sunny' : 0.8, 'rainy' : 0.2}
rainy_transition = {'sunny' : 0.4, 'rainy' : 0.6}
transition_prob = {'sunny': sunny_transition, 'rainy': rainy_transition}
print(f"transition probability : {transition_prob}")

# probabilidades de emisiones
sunny_transition = {'sunny' : 0.8, 'rainy' : 0.2}
rainy_transition = {'sunny' : 0.3, 'rainy' : 0.7}
emission_prob = {'sunny': sunny_transition, 'rainy': rainy_transition}
print(f"emission probability : {emission_prob}")

observations : ['sunny', 'sunny', 'rainy']
initial probability : {'sunny': 0.5, 'rainy': 0.5}
transition probability : {'sunny': {'sunny': 0.8, 'rainy': 0.2}, 'rainy': {'sunny': 0.4, 'rainy': 0.6}}
emission probability : {'sunny': {'sunny': 0.8, 'rainy': 0.2}, 'rainy': {'sunny': 0.3, 'rainy': 0.7}}


## Creacion del Forward Backward y HMM

In [45]:
import random

class HMM:
    def __init__(self, states=None, observations=None, initial_prob=None, transition_prob=None, emission_prob=None):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob
    

    def forward(self, observations):
        # se inicializa alpha como un diccionario que contendrá la info de las probabilidades siguientes
        alpha = [{}]
        # Caso base para Forward
        for state in self.states:
            # self.initial_prob[state] : probabilidad inicial dado el estado
            # self.emission_prob[state][observations[0]] : probabilidad de emisión dado el estado y primera observación
            alpha[0][state] = self.initial_prob[state] * self.emission_prob[state][observations[0]]

        # Caso para calcular Forward dado el número de observaciones
        # Se inicia desde la segunda observación hasta la útlima
        # alpha[t-1][previous_state] : probabilidad previa del time-step para el estado anterior
        # self.transition_prob[previous_state][current_state] : probabilidad de pasar del antrior estado al actual
        # self.emission_prob[current_state][observations[t]] : probabilidad de observar la observación actual dado el estado actual
        for t in range(1, len(observations)):
            alpha.append({})
            for current_state in self.states:
                alpha[t][current_state] = sum(
                    alpha[t-1][previous_state] * self.transition_prob[previous_state][current_state] *
                    self.emission_prob[current_state][observations[t]] for previous_state in self.states
                )
        return alpha

    def backward(self, observations):
        # se inicializa diccionario beta con longitud de observaciones
        # cada diccionario corresponde a un time-step y contendrá la probabilidad retrógrada de ese time-step
        # asumimos que la última probabilidad es 1
        beta = [{} for _ in range(len(observations))]
        # Caso base para el Backward
        for state in self.states:
            beta[len(observations)-1][state] = 1

        # Caso recursivo para el backward step
        # el bucle itera en retroceso desde la penúltma observación a la primera
        # beta[t+1][future_state] : la probabilidad retrógada en t+1
        # self.transition_prob[state][future_state] : probabilidad de transitar del estado actual al siguiente
        # self.emission_prob[future_state][observations[t+1]] : probabilidad de observar la siguiente observación dado el estado futuro
        for t in range(len(self.observations) - 2, -1, -1):
            for state in self.states:
                beta[t][state] = sum(
                    beta[t+1][future_state] * self.transition_prob[state][future_state] *
                    self.emission_prob[future_state][observations[t+1]] for future_state in self.states
                )
        # retornar las probabilidades en retrógradas
        return beta

    def generate_sequence(self, length=10):
        # Empezar con un estado inicial de las probabilidades iniciales
        states_sequence = [random.choices(self.states, weights=self.initial_prob.values())[0]]
        observations_sequence = []

        # Generar secuencias de estados
        for i in range(1, length):
            # Obtener la probabilidad del último estado
            last_state = states_sequence[-1]
            current_state = random.choices(
                self.states,
                weights=[self.transition_prob[last_state][next_state] for next_state in self.states]
            )[0]
            states_sequence.append(current_state)

        # Generar las observaciones badasdo en la secuencia de estados
        for state in states_sequence:
            observation = random.choices(
                self.observations,
                weights=[self.emission_prob[state][obs] for obs in self.observations]
            )[0]
            observations_sequence.append(observation)

        return observations_sequence

    def compute_state_probabilities(self, observations):
        # estimar probabilidades forward y backward
        forward_probs = self.forward(observations)
        backward_probs = self.backward(observations)
        state_probs = []
        # se itera en el número de observaciones
        for t in range(len(observations)):
            state_probs_t = {}
            total_prob_t = sum(
                forward_probs[t][state] * backward_probs[t][state] for state in self.states
            )
            for state in self.states:
                state_probs_t[state] = (
                    forward_probs[t][state] * backward_probs[t][state] / total_prob_t
                )
            state_probs.append(state_probs_t)
        return state_probs

hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)
observations_hmm = hmm.generate_sequence(length=5)
print(f"Observations with HMM : {observations_hmm}")

forward = hmm.forward(observations)
print(f"forward : {forward}")

backward = hmm.backward(observations)
print(f"Backward : {backward}")

state_probabilties = hmm.compute_state_probabilities(observations)
# muestra la probabilidad que hay en cada estado 
print(f"Probabilities : {state_probabilties}")


Observations with HMM : ['sunny', 'rainy', 'sunny', 'sunny', 'sunny']
forward : [{'sunny': 0.4, 'rainy': 0.15}, {'sunny': 0.30400000000000005, 'rainy': 0.051000000000000004}, {'sunny': 0.05272000000000002, 'rainy': 0.06398000000000001}]
Backward : [{'sunny': 0.22200000000000006, 'rainy': 0.18600000000000003}, {'sunny': 0.30000000000000004, 'rainy': 0.5}, {'sunny': 1, 'rainy': 1}]
Probabilities : [{'sunny': 0.7609254498714653, 'rainy': 0.23907455012853465}, {'sunny': 0.7814910025706941, 'rainy': 0.21850899742930588}, {'sunny': 0.45175664095972584, 'rainy': 0.5482433590402742}]
